# Task 2: Assign Respond Tone for Target Character's Lines

In [27]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from tqdm import tqdm

In [31]:
# read in csv
df_chigiri = pd.read_csv('data/chigiri_pairs_new.csv')

In [2]:
# load BERT tokeizeer and model
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese", use_fast=False)
bert_model = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese")
bert_model.eval()

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [3]:
def get_bert_embedding(text):
    """
    Get the bert embedding of the given text. Use mean pooling method to 
    keep the embedding vector the same size.
    
    Args:
        text: Japanese text to be embedded
        
    Return:
        embedded_vector: 768-d embedded_vector of a sentence
    """
    with torch.no_grad():
        # tokenize and get embedding vector for each token
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
        outputs = bert_model(**inputs)
        # [batch_size, seq_len, hidden_size]
        hidden_states = outputs.last_hidden_state  
        
        # identify padding and make it into 0
        mask = inputs["attention_mask"].unsqueeze(-1).expand(hidden_states.size())
        masked_embeddings = hidden_states * mask
        
        # do mean pooling to get an average of the tokens
        # having 768-d embedding vectors for all sentences
        summed = torch.sum(masked_embeddings, dim=1)
        counts = torch.clamp(mask.sum(dim=1), min=1e-9)
        mean_pooled = summed / counts
    return mean_pooled.squeeze().numpy()

def sentence_embedding(df, text_column="chigiri_line", output_column="embedding"):
    """
    Embedding function for a whole dataframe. Add a new column for embedding vectors.
    
    Args:
        df: dataframe with text 
        text_column: column name of where text existed
        output_column: column name of where the embedded vectors should be
        
    Return:
        df: updated dataframe with embedded vecotr.
    """
    embeddings = []
    for text in tqdm(df[text_column]):
        if pd.isna(text) or text.strip() == "":
            embeddings.append(None)
        else:
            emb = get_bert_embedding(text)
            embeddings.append(emb)
    df[output_column] = embeddings
    return df

In [32]:
# do embedding for chigiri's dataset
df_chigiri = sentence_embedding(df_chigiri, text_column="chigiri_line")

100%|█████████████████████████████████████████| 217/217 [00:08<00:00, 26.99it/s]


In [33]:
# save the result for backup
# df_chigiri.to_pickle("chigiri_with_embedding.pkl")

In [34]:
from sklearn.cluster import KMeans
import numpy as np

# get out all embedding vectors
valid_embeddings = df_chigiri["embedding"].dropna().tolist()
X = np.stack(valid_embeddings)

# seperate them into 10 clusters
kmeans = KMeans(n_clusters=10, random_state=42)
df_chigiri.loc[df_chigiri["embedding"].notna(), "tone_cluster"] = kmeans.fit_predict(X)


In [35]:
# print out the first 10 sentences to check
for i in range(10): 
    print(f"\n🟦 Cluster {i} 語氣句子:")
    display(df_chigiri[df_chigiri["tone_cluster"] == i]["chigiri_line"].head(10))


🟦 Cluster 0 語氣句子:


81    ロック･オフ ロック･オフ… ロック･オフ
Name: chigiri_line, dtype: object


🟦 Cluster 1 語氣句子:


36                          うるせえ 黙れ！
37                               黙れ！
49                 どけ ぶち抜け！ この滾りに従え！
54     見ろ <name>！　見ろ お前ら！ これが 千切豹馬だ！
59                  ああ やってみろ！世界一は俺だ！
65                             なんか違う
68    させるかよ！今度は俺が潰すスピードキングは俺だ、バカメガネ！
69          やられっぱなしで終われるかよ！ 届いたぞのろま！
70                           お嬢じゃねえよ
83                        させるか！切り返し？
Name: chigiri_line, dtype: object


🟦 Cluster 2 語氣句子:


...

47           出せ <name>！
87              <name>！
106             <name>？
111        <name>はどう思う？
128    ‪ナイッシュー <name>！‬
183         突っ込め<name>！
198    <name>だけ斜めに突破した？
Name: chigiri_line, dtype: object


🟦 Cluster 8 語氣句子:


0             じゃあ ここにいるチームＺ 11人が１つのチームってこと？ 全員フォワードなのに？
2     俺は　あいつの言ってることが全部正しいとは思ってないよ。 だって 絵心はワールドカップ優勝す...
6       っていうか、この作戦で大丈夫なの？ それぞれの武器が通用しなかったら、このままずるずる負けるよ
7     勝つ確率、考えたら 使えない武器で時間をムダにするより、蜂楽か國神みたいな相手の嫌がる攻撃を...
11                  そりゃ寝られないよな こんなの決めたらストライカーとして最高の瞬間だろ
12                  ブルーロックに来る前の俺だったら絶対味わえないような ゴールだったから
13                                    空間認識能力が高いんだよ多分 お前
14    ほとんどの選手はみんな自分の視野から状況を判断してプレーしている トップスピードで走ってると...
15    お前はまだそれを直感でやってるけど、もっと意識的に使えるようになればその目と脳は唯一無二の武...
16                                  別に…チームが勝つために言っただけだし
Name: chigiri_line, dtype: object


🟦 Cluster 9 語氣句子:


34     クソ… クソッ！
210    クソ… クソッ！
Name: chigiri_line, dtype: object

In [36]:
# assign name for each cluster
dict_group_tone = {
    0: "テンション高めにふざけて・ネタっぽく", 
    1: "怒っている・キレている・強気で挑発的に",
    2: "言いたくない・感情を隠す・葛藤をにじませる", 
    3: "普通の会話、軽く返す・冷静にリアクション", 
    4: "皮肉っぽく・ぶっきらぼう・子供っぽく拗ねて", 
    5: "悔しさをにじませつつ本音で熱く", 
    6: "焦り・動揺・緊張した様子で", 
    7: "短く叫ぶ・反応を返す・テンポ速く強めに",
    8: "冷静に説明する・論理的に・分析的に",
    9: "感情爆発・強く叫ぶ・悔しさや怒りをぶつける", 
}

In [37]:
def assign_tone(df, cluster_to_tone, cluster_column="tone_cluster", output_column="chigiri_tone"):
    """
    Update the assigned tone to the original dataframe.
    
    Args:
        df: dataframe with text after clustering
        cluster_to_tone (dict): assigned name for each cluster
        cluster_column: name of the column where the cluster information existed
        output_column: name fo the new column
    
    """
    df[output_column] = df[cluster_column].map(cluster_to_tone)
    return df

df_chigiri_tone = assign_tone(df_chigiri, dict_group_tone)

In [38]:
df_chigiri_tone.head()

,Unnamed: 0,prev_line,chigiri_line,embedding,tone_cluster,chigiri_tone
0,0,一次セレクションはお前らのいる伍号棟 55名全５チームによる、総当たりリーグ戦 上位２チーム...,じゃあ ここにいるチームＺ 11人が１つのチームってこと？ 全員フォワードなのに？,"[0.32861152, 0.10151904, 0.40979138, -0.112925...",8.0,冷静に説明する・論理的に・分析的に
1,1,勝ちゃあいいんだろ 勝ちゃあ,勝ちゃあいい… ねえ,"[-0.044169173, 0.2113115, 0.053375773, -0.2689...",6.0,焦り・動揺・緊張した様子で
2,2,つうか やっぱチーム全員が フォワードって意味分かんねえ,俺は あいつの言ってることが全部正しいとは思ってないよ。 だって 絵心はワールドカップ優勝す...,"[0.2839429, 0.06686456, 0.33462584, -0.4077513...",8.0,冷静に説明する・論理的に・分析的に
3,3,何してんの？ 千切,髪のケア,"[0.16518353, -0.04691266, 0.33033556, -0.18144...",3.0,普通の会話、軽く返す・冷静にリアクション
4,4,ラスト 千切は？,言いたくない,"[0.054298688, -0.24455349, 0.06226331, 0.14399...",2.0,言いたくない・感情を隠す・葛藤をにじませる


In [39]:
df_chigiri_tone = df_chigiri_tone.drop(columns=["embedding"])
df_chigiri_tone.to_csv("data/chigiri_train_w_tone_.csv")

Then, I manually check the assigned tone to make sure there isn't huge mistake.